In [1]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from datetime import datetime
import string

^ We are importing all the necessary packages in order to execute this code.

In [2]:
sc = SparkContext(appName="PythonStreamingQueueStream")    

^ We are creating a SparkContext here.

In [3]:
sqlContext = SQLContext(sc)

^ And here SqlContext.

In [4]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('D:/Nisum/Employees.csv')
df.show(5)

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567| 17-06-03|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568| 21-09-05|  AD_VP| 17000|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569| 13-01-01|  AD_VP| 17000|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567| 03-01-06|IT_PROG|  9000|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568| 21-05-07|IT_PROG|  6000|          null|       103|           60|
+-----------+----------+

^ We are reading a csv file "Employees.csv", using sqlContext and saving it into "df"

In [ ]:
df.printSchema()

^ As you can see in the schema of df, HIRE_DATE is of type string.

In [ ]:
df1=df.withColumn("HIRE_DATE",(to_date(df["HIRE_DATE"],'dd-MM-yy').alias('HIRE_DATE')).cast(DateType()))
df1.show(5)

^ In the above cell, we are selecting "HIRE_DATE" column in which the date representaion is in "dd-MM-yy", and performing to_date operations on it and casting it int DateType

In [ ]:
df1.printSchema()

^ When you observe in the schema, HIRE_DATE is of *date* type

In [9]:
def dateCustomFormat(dataFrame, col, curr_date_format, req_date_format):
    data = dataFrame.withColumn(col,from_unixtime(unix_timestamp(dataFrame[col], curr_date_format),req_date_format).alias(col))
    return data

^ we are able to define a function, which take dataFrame, col, and date_format as arguments, and perform the to_date function on it to convert a string type column into a date type column.

In [11]:
df2 = dateCustomFormat(df,"HIRE_DATE","dd-MM-yy","yyyyMM")
df2.show(5)

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|   200306|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|   200509|  AD_VP| 17000|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|   200101|  AD_VP| 17000|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|   200601|IT_PROG|  9000|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|   200705|IT_PROG|  6000|          null|       103|           60|
+-----------+----------+

^ we are passing df, "HIRE_DATE", and "dd-MM-yy" as paramaters. The returned data is stored into df2

In [ ]:
df2.printSchema()

^ HIRE_DATE is changed to date type

In [ ]:
hire_date = df.select("HIRE_DATE").collect()

In [ ]:
type(hire_date)

In [ ]:
data = ['01/01/2012 01:00', '01/01/2012 02:00', '01/01/2012 03:00', '01/01/2012 04:00', '01/01/2012 05:00',
        '01/01/2012 06:00', '01/01/2012 07:00', '01/01/2012 08:00', '01/01/2012 09:00', '01/01/2012 10:00',
        '01/01/2012 11:00', '01/01/2012 12:00', '01/01/2012 13:00', '01/01/2012 14:00', '01/01/2012 15:00',
        '01/01/2012 16:00', '01/01/2012 17:00', '01/01/2012 18:00', '01/01/2012 19:00', '01/01/2012 20:00', 
        '01/01/2012 21:00', '01/01/2012 22:00', '01/01/2012 23:00', '02/01/2012 00:00', '04/01/2012 23:00']

^ we manually typed some dates in a list and assigned it to data variable

In [ ]:
dates = list(map(lambda x: datetime.strptime(x,  "%m/%d/%Y %H:%M").strftime("%Y-%m-%d %I:%M:%S"), hire_date))
print(dates)

In [ ]:
type(dates)

In [ ]:
data1 = list(map(lambda x: datetime.strptime(x,  "%m/%d/%Y %H:%M").strftime("%Y/%m/%d %I:%M:%S"), data))
print(data1)

In [ ]:
oldformat = df2["HIRE_DATE"]
datetimeobject = datetime.strptime(oldformat,'%d-%m-%y')